In [19]:
#Importing packages:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import constants

In [29]:
# Calculating radius for collapse:
Mass=1e11
Gpc=4.3e-6

Ho = 70e-3  # [km/kpc/s] Hubble constant
lamM = 0.26  # Matter density 
lamL = 1-lamM # Dark Energy density
zinit = 15 # Starting density
zcol = 4 # Approximate collapse redshift (from linear theory)
ainit = 1./(1.+zinit) # a is the expansion factor
acol = 1./(1.+zcol)  # a is defined as 1/(1+z)
RHOcrit = 3.*(Ho**2)/(8.*pi*Gpc) # average density of Universe from Friedmann’s equation
Hinit = Ho*((lamM/ainit**3)-lamL)**0.5 # the value of Hubble constant at the initial redshift.
RHOcrit_init = (3*Hinit**2)/(8*pi*Gpc) # from Friedmann’s equation as above
LAMinit = 1.686*ainit/acol # the “overdensity” [(RHO-RHOave)/RHOave] required to collapse.
RHOcol = RHOcrit_init*(LAMinit+1) # alter RHOcrit_init by the amount suggested by linear theory
rcol = (3*Mass/(4*pi*RHOcol))**(1./3.) # radius of collapse... re-arranged Density = Mass/Vol

In [30]:
# Setting up a sphere of 1000 particles without Hubble flow:

# Create Distribution

import random
import numpy as np

random.seed(77)

npart = 1000
Mtotal = 1e11 #10e4msun
rmax = rcol #kpc
Vtot = 4*np.pi*(rmax**3)/3 #kpc^3
n = npart/(Vtot) #kpc^-3
meanlen = (1/n)**(1/3) #kpc

soft = meanlen/30

sall = 1

phi = np.random.uniform(0,2*np.pi,npart)
costheta = np.random.uniform(-1,1,npart)
u = np.random.uniform(0,1,npart)

theta = np.arccos( costheta )
r = rmax *u**(1/3)

x = r * np.sin( theta) * np.cos( phi )
y = r * np.sin( theta) * np.sin( phi )
z = r * np.cos( theta )

ID = np.linspace(1,npart,npart,dtype = int)

x.shape = (npart,1)
y.shape = (npart,1)
z.shape = (npart,1)
Coordinates = np.concatenate((x,y,z),axis = 1)
Velocities = np.zeros_like(Coordinates)

#Create hdf5

import h5py
import sys

filename = 'CDM_ICs.hdf5'

!rm SphereIC.hdf5

#Create File
file = h5py.File(filename,'w')

#Create Group
header = file.create_group("Header")
PartType1 = file.create_group("PartType1")

coordsDSET = file.create_dataset("/PartType1/Coordinates", (npart,3), dtype =  h5py.h5t.IEEE_F32LE)
IDsDSET = file.create_dataset("/PartType1/ParticleIDs", (npart,), dtype =   h5py.h5t.STD_U32LE)
velsDSET = file.create_dataset("/PartType1/Velocities", (npart,3), dtype =  h5py.h5t.IEEE_F32LE)

coordsDSET[...] = Coordinates
IDsDSET[...] = ID
velsDSET[...] = Velocities

nparts = np.zeros(3)
nparts[1] = npart
header.attrs['NumPart_ThisFile']    = nparts

massvec = np.zeros(3)
massvec[1] = Mtotal/npart

header.attrs['MassTable']           = massvec
header.attrs['Time']                = 0
header.attrs['Redshift']            = 15
header.attrs['NumPart_Total']       = nparts
header.attrs['NumFilesPerSnapshot'] = 1
header.attrs['BoxSize']             = 1.0
header.attrs['Omega0']              = 1.0
header.attrs['OmegaLambdda']        = 0.
header.attrs['HubbleParam']         = 0.7
header.attrs['Flag_Entropy_ICs']    = 0
header.attrs['NumPart_Total_HighWord'] = np.zeros(3)


# Close the file before exiting

file.close()

rm: cannot remove 'SphereIC.hdf5': No such file or directory


In [31]:
# Setting up a sphere of 1000 particles WITH Hubble flow:

# Create Distribution

import random
import numpy as np

random.seed(77)

npart = 1000
Mtotal = 1e11 #10e4msun
rmax = rcol #kpc
Vtot = 4*np.pi*(rmax**3)/3 #kpc^3
n = npart/(Vtot) #kpc^-3
meanlen = (1/n)**(1/3) #kpc

soft = meanlen/30

sall = 1

phi = np.random.uniform(0,2*np.pi,npart)
costheta = np.random.uniform(-1,1,npart)
u = np.random.uniform(0,1,npart)

theta = np.arccos( costheta )
r = rmax *u**(1/3)

x = r * np.sin( theta) * np.cos( phi )
y = r * np.sin( theta) * np.sin( phi )
z = r * np.cos( theta )

ID = np.linspace(1,npart,npart,dtype = int)

x.shape = (npart,1)
y.shape = (npart,1)
z.shape = (npart,1)
Coordinates = np.concatenate((x,y,z),axis = 1)

#Calculating Hubble flow for initial velocities:
H=Hinit*(1.-LAMinit/3.)
Hx,Hy,Hz=H*x,H*y,H*z
Hx.shape = (npart,1)
Hy.shape = (npart,1)
Hz.shape = (npart,1)
Velocities = np.concatenate((Hx,Hy,Hz),axis = 1)

#Create hdf5
filename = 'CDM_ICs_HF.hdf5'

#Create File
file = h5py.File(filename,'w')

#Create Group
header = file.create_group("Header")
PartType1 = file.create_group("PartType1")

coordsDSET = file.create_dataset("/PartType1/Coordinates", (npart,3), dtype =  h5py.h5t.IEEE_F32LE)
IDsDSET = file.create_dataset("/PartType1/ParticleIDs", (npart,), dtype =   h5py.h5t.STD_U32LE)
velsDSET = file.create_dataset("/PartType1/Velocities", (npart,3), dtype =  h5py.h5t.IEEE_F32LE)

coordsDSET[...] = Coordinates
IDsDSET[...] = ID
velsDSET[...] = Velocities

nparts = np.zeros(3)
nparts[1] = npart
header.attrs['NumPart_ThisFile']    = nparts

massvec = np.zeros(3)
massvec[1] = Mtotal/npart

header.attrs['MassTable']           = massvec
header.attrs['Time']                = 0
header.attrs['Redshift']            = 15
header.attrs['NumPart_Total']       = nparts
header.attrs['NumFilesPerSnapshot'] = 1
header.attrs['BoxSize']             = 1.0
header.attrs['Omega0']              = 1.0
header.attrs['OmegaLambdda']        = 0.
header.attrs['HubbleParam']         = 0.7
header.attrs['Flag_Entropy_ICs']    = 0
header.attrs['NumPart_Total_HighWord'] = np.zeros(3)


# Close the file before exiting
#
file.close()

In [32]:
# Setting up a sphere of 1000 particles WITH Hubble flow:

# Create Distribution

import random
import numpy as np

random.seed(77)

npart = 10000
Mtotal = 1e11 #10e4msun
rmax = rcol #kpc
Vtot = 4*np.pi*(rmax**3)/3 #kpc^3
n = npart/(Vtot) #kpc^-3
meanlen = (1/n)**(1/3) #kpc

soft = meanlen/30

sall = 1

phi = np.random.uniform(0,2*np.pi,npart)
costheta = np.random.uniform(-1,1,npart)
u = np.random.uniform(0,1,npart)

theta = np.arccos( costheta )
r = rmax *u**(1/3)

x = r * np.sin( theta) * np.cos( phi )
y = r * np.sin( theta) * np.sin( phi )
z = r * np.cos( theta )

ID = np.linspace(1,npart,npart,dtype = int)

x.shape = (npart,1)
y.shape = (npart,1)
z.shape = (npart,1)
Coordinates = np.concatenate((x,y,z),axis = 1)

#Calculating Hubble flow for initial velocities:
H=Hinit*(1.-LAMinit/3.)
Hx,Hy,Hz=H*x,H*y,H*z
Hx.shape = (npart,1)
Hy.shape = (npart,1)
Hz.shape = (npart,1)
Velocities = np.concatenate((Hx,Hy,Hz),axis = 1)

#Create hdf5
filename = 'CDM_ICs_HF_n10000.hdf5'

#Create File
file = h5py.File(filename,'w')

#Create Group
header = file.create_group("Header")
PartType1 = file.create_group("PartType1")

coordsDSET = file.create_dataset("/PartType1/Coordinates", (npart,3), dtype =  h5py.h5t.IEEE_F32LE)
IDsDSET = file.create_dataset("/PartType1/ParticleIDs", (npart,), dtype =   h5py.h5t.STD_U32LE)
velsDSET = file.create_dataset("/PartType1/Velocities", (npart,3), dtype =  h5py.h5t.IEEE_F32LE)

coordsDSET[...] = Coordinates
IDsDSET[...] = ID
velsDSET[...] = Velocities

nparts = np.zeros(3)
nparts[1] = npart
header.attrs['NumPart_ThisFile']    = nparts

massvec = np.zeros(3)
massvec[1] = Mtotal/npart

header.attrs['MassTable']           = massvec
header.attrs['Time']                = 0
header.attrs['Redshift']            = 15
header.attrs['NumPart_Total']       = nparts
header.attrs['NumFilesPerSnapshot'] = 1
header.attrs['BoxSize']             = 1.0
header.attrs['Omega0']              = 1.0
header.attrs['OmegaLambdda']        = 0.
header.attrs['HubbleParam']         = 0.7
header.attrs['Flag_Entropy_ICs']    = 0
header.attrs['NumPart_Total_HighWord'] = np.zeros(3)


# Close the file before exiting

file.close()